# Review of julia itterative solver


![ssor_image](algorithm_4.png)

In [ ]:
"""
struct SparseMatrixCSC{Tv,Ti<:Integer} <: AbstractSparseMatrix{Tv,Ti}
    m::Int                  # Number of rows
    n::Int                  # Number of columns
    colptr::Vector{Ti}      # Column i is in colptr[i]:(colptr[i+1]-1)
    rowval::Vector{Ti}      # Row indices of stored values
    nzval::Vector{Tv}       # Stored values, typically nonzeros
end
"""


In [84]:
  
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate

using SparseArrays

struct DiagonalIndices{Tv, Ti <: Integer}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::Vector{Ti}

    function DiagonalIndices{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti}
        # Check square?
        diag = Vector{Ti}(undef, A.n)

        for col = 1 : A.n
            r1 = Int(A.colptr[col])
            r2 = Int(A.colptr[col + 1] - 1)
            r1 = searchsortedfirst(A.rowval, col, r1, r2, Base.Order.Forward)
            if r1 > r2 || A.rowval[r1] != col || iszero(A.nzval[r1])
                throw(LinearAlgebra.SingularException(col))
            end
            diag[col] = r1
        end 

        new(A, diag) #
    end
end

DiagonalIndices(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti} = DiagonalIndices{Tv,Ti}(A)
#밖에서도 함수로 쓸수있게 따로 빼준건가?

#DiagonalIndices 특징은 Sparse matrix의 nz diagonal element vector로 변환시키며,
#diagonal term에 0이 있으면 안된다.
#원 matrix을 담고 있는 구조체임.
# A.nz는 col 순서대로 적어내린 values
# A.row는 nz 기준 해당 row index
# A.cor은 i에  해당하는 nz index 범위를 찍어줌






DiagonalIndices

In [90]:

struct FastLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end


In [96]:
D_a = DiagonalIndices(A)
LT = FastLowerTriangular(D_a.matrix,D_a)

FastLowerTriangular{Int64,Int64}(
  [1, 1]  =  1
  [4, 1]  =  1
  [1, 2]  =  2
  [2, 2]  =  1
  [3, 2]  =  1
  [1, 3]  =  3
  [3, 3]  =  4
  [3, 4]  =  1
  [4, 4]  =  10, DiagonalIndices{Int64,Int64}(
  [1, 1]  =  1
  [4, 1]  =  1
  [1, 2]  =  2
  [2, 2]  =  1
  [3, 2]  =  1
  [1, 3]  =  3
  [3, 3]  =  4
  [3, 4]  =  1
  [4, 4]  =  10, [1, 4, 7, 9]))

In [78]:
"""
Forward substitution for the FastLowerTriangular type
"""
function forward_sub!(F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix

    @inbounds for col = 1 : A.n

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # 여기까진 오케 idx는 diag 까지 카운트된 idx
        display(x)

        # Substitute next values involving x[col]
        for i = idx + 1 : (A.colptr[col + 1] - 1)  #해당 diag부터 그다음 col 전까지 nz index
            #roval[i]는 해당하는 row를 반환 x[row] = x[row] - A해당하는거 곱하고 *를 
            x[A.rowval[i]] -= A.nzval[i] * x[col]
        end
    end

    x
end

forward_sub!

# miscellaneous

In [86]:
A_ = sparse([1 0 3 0; 2 1 0 0;0 1 4 1; 0 0 1 10])
Da_ = DiagonalIndices(A_)

display("matrix A_")
display(Matrix(A_))
display("matrix A_ diag")
display(Da_.diag)

A = sparse([1 2 3 0; 0 1 0 0;0 1 4 1; 1 0 0 10])
Da = DiagonalIndices(A)

display("matrix A")
display(Matrix(A))
display("matrix A diag")
display(Da.diag)


#A.colptr #col_i에서 매칭된 element 개수와 위치 with A.nzval[col[i] : col[i+1]-1]
#A.diag는 col base counting에서 n-th diag elment까지 nz element의 개수(자기 포함)
#diag는 nzval map을 이용해서 diag로 접근가능
#get index 참고하면 알 수 있음.

"matrix A_"

4×4 Array{Int64,2}:
 1  0  3   0
 2  1  0   0
 0  1  4   1
 0  0  1  10

"matrix A_ diag"

4-element Array{Int64,1}:
 1
 3
 6
 9

"matrix A"

4×4 Array{Int64,2}:
 1  2  3   0
 0  1  0   0
 0  1  4   1
 1  0  0  10

"matrix A diag"

4-element Array{Int64,1}:
 1
 4
 7
 9

In [74]:
A.rowval

9-element Array{Int64,1}:
 1
 4
 1
 2
 3
 1
 3
 3
 4

The following two types are equivalent – one with a default constructor, the other with an explicit constructor:

~~~
julia> struct T1
           x::Int64
       end

julia> struct T2
           x::Int64
           T2(x) = new(x)
       end
~~~
